# Concurrent Fan-Out/Fan-In Orchestration

## Overview

This notebook demonstrates **concurrent orchestration** using the `ConcurrentBuilder` API. Instead of executing agents sequentially, this pattern fans out a single user prompt to multiple domain experts simultaneously, then fans in their responses into a single aggregated output.

### Key Concepts:

1. **ConcurrentBuilder**: High-level API for building parallel workflows
2. **Fan-Out Pattern**: Same prompt dispatched to multiple agents in parallel
3. **Fan-In Aggregation**: Results collected and combined into final output
4. **Default Aggregator**: Returns `list[ChatMessage]` with all conversations
5. **Workflow Completion**: Finishes when idle with no pending work

### Architecture:

```
                    User Prompt
                        ↓
            [Internal Dispatcher]
               ↙       ↓        ↘
         Researcher  Marketer  Legal
         (parallel execution)
               ↘       ↓        ↙
            [Default Aggregator]
                        ↓
            Aggregated Output
     (list[ChatMessage] from all agents)
```

### When to Use Concurrent Orchestration:

- **Multi-perspective analysis**: Need insights from different domain experts
- **Parallel processing**: Tasks that can be executed independently
- **Speed optimization**: Reduce total execution time vs. sequential
- **Comprehensive coverage**: Gather diverse viewpoints simultaneously

## Prerequisites

- Azure OpenAI configured with environment variables:
  - `AZURE_OPENAI_API_KEY`
  - `AZURE_OPENAI_ENDPOINT`
  - `AZURE_OPENAI_DEPLOYMENT_NAME`
  - `AZURE_OPENAI_API_VERSION`
- Azure CLI authentication: Run `az login` before executing
- Agent Framework installed: `pip install agent-framework`

## Setup and Imports

In [ ]:
import asyncio
from typing import Any

from agent_framework import ChatMessage, ConcurrentBuilder
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


## Create Domain Expert Agents

We'll create three specialized agents, each with distinct expertise:

### 1. Researcher
- Provides factual market and product insights
- Analyzes opportunities and risks
- Data-driven perspective

### 2. Marketer
- Crafts value propositions
- Develops target messaging
- Creative and persuasive angle

### 3. Legal
- Highlights constraints and disclaimers
- Identifies compliance requirements
- Risk mitigation focus

In [ ]:
async def run_concurrent_workflow() -> None:
    # Create Azure OpenAI chat client with Azure CLI authentication
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    researcher = chat_client.create_agent(
        instructions=(
            "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
            " opportunities, and risks."
        ),
        name="researcher",
    )

    marketer = chat_client.create_agent(
        instructions=(
            "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
            " aligned to the prompt."
        ),
        name="marketer",
    )

    legal = chat_client.create_agent(
        instructions=(
            "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
            " based on the prompt."
        ),
        name="legal",
    )

## Build Concurrent Workflow

### Key Points:

- **`.participants([...])`**: Accepts agents (AgentProtocol) or Executors
- **Minimal wiring**: No need to define edges manually
- **Internal dispatcher**: Automatically fans out to all participants
- **Default aggregator**: Collects final ChatMessages from all agents

In [ ]:
    # Build concurrent workflow with all three agents
    workflow = ConcurrentBuilder().participants([researcher, marketer, legal]).build()

## Execute Workflow and Display Results

The workflow will:
1. Send the same prompt to all three agents simultaneously
2. Wait for all agents to complete their responses
3. Aggregate results into a single output
4. Return `list[ChatMessage]` containing all conversations

In [ ]:
    # Run workflow with a single prompt
    events = await workflow.run("We are launching a new budget-friendly electric bike for urban commuters.")
    outputs = events.get_outputs()

    if outputs:
        print("===== Final Aggregated Conversation (messages) =====")
        for output in outputs:
            messages: list[ChatMessage] | Any = output
            for i, msg in enumerate(messages, start=1):
                name = msg.author_name if msg.author_name else "user"
                print(f"{'-' * 60}\n\n{i:02d} [{name}]:\n{msg.text}")

## Run the Complete Workflow

In [ ]:
await run_concurrent_workflow()

## Expected Output

```
===== Final Aggregated Conversation (messages) =====
------------------------------------------------------------

01 [user]:
We are launching a new budget-friendly electric bike for urban commuters.
------------------------------------------------------------

02 [researcher]:
**Insights:**

- **Target Demographic:** Urban commuters seeking affordable, eco-friendly transport;
    likely to include students, young professionals, and price-sensitive urban residents.
- **Market Trends:** E-bike sales are growing globally, with increasing urbanization,
    higher fuel costs, and sustainability concerns driving adoption.
- **Competitive Landscape:** Key competitors include brands like Rad Power Bikes, Aventon,
    Lectric, and domestic budget-focused manufacturers in North America, Europe, and Asia.
...
------------------------------------------------------------

03 [marketer]:
**Value Proposition:**
"Empowering your city commute: Our new electric bike combines affordability, reliability, and
    sustainable design—helping you conquer urban journeys without breaking the bank."
...
------------------------------------------------------------

04 [legal]:
**Constraints, Disclaimers, & Policy Concerns:**

**1. Regulatory Compliance**
- Verify that the electric bike meets all applicable federal, state, and local regulations
    regarding e-bike classification, speed limits, power output, and safety features.
...
```

## Key Takeaways

### 1. Concurrent Builder Simplicity
- Single line to create workflow: `ConcurrentBuilder().participants([...]).build()`
- No manual edge definition required
- Automatic dispatcher and aggregator setup

### 2. Parallel Execution Benefits
- **Faster results**: All agents run simultaneously
- **Independent analysis**: Each agent provides unique perspective
- **Efficient resource use**: No waiting for sequential completion

### 3. Default Aggregation
- Returns `list[ChatMessage]` with:
  - One user message (original prompt)
  - One assistant message per agent (their responses)
- Preserves `author_name` for identifying which agent responded

### 4. Workflow Completion
- Completes when all participants are idle
- No pending messages to process
- Outputs accessible via `events.get_outputs()`

### 5. Use Cases
- **Product launches**: Research + Marketing + Legal perspectives
- **Risk assessment**: Multiple domain experts evaluating scenarios
- **Content creation**: Diverse viewpoints for comprehensive coverage
- **Decision support**: Parallel analysis from different angles

### 6. Agent vs. Executor Participants
- Can mix **AgentProtocol** (agents) and **Executor** instances
- Agents use default agent executor wrapper
- Custom executors provide more control (see concurrent_custom_agent_executors.py)

### 7. Customization Options
- **Custom aggregator**: Override default with `.with_aggregator(callback)` (see concurrent_custom_aggregator.py)
- **Custom dispatcher**: Can replace default dispatcher logic
- **Hybrid patterns**: Combine concurrent and sequential stages

### 8. Production Considerations
- Monitor total token usage across all parallel agents
- Consider rate limits when scaling to many participants
- Add error handling for individual agent failures
- Log agent responses separately for debugging
- Consider timeout handling for slow agents